# Strategy and Performance Management - Fall 2022 - Group Project
## H&M Customer Segmentation and Strategies
### Group 6: Thi Anh Ba Dang, Eduardo Garcia, Thu Lam, Carla Weidner

---



# Exploratory Data Analysis

## Importing Data from Kaggle to Colab

In [ ]:
# upload API's key JSON file to 

from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations --force

In [ ]:
!unzip -qq h-and-m-personalized-fashion-recommendations.zip


In [ ]:
import shutil
shutil.rmtree('images')

## Import and Load Libraries

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import label encoder
from sklearn import preprocessing

# import one hot encoder
from sklearn.preprocessing import OneHotEncoder

## Cleaning customers.csv

This dataset has seven (7) features and 1,371,980 records.

In [45]:
# Load the data
df_customers = pd.read_csv('/Users/mythulam/Desktop/Masters/03_Fall_2022/02_Strategy_and_Performance_Management/Session_10_Group_Project/Data/customers.csv')

In [46]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [47]:
# data cleaning
df_customers['FN'] = df_customers['FN'].fillna(0)   #fill nulls/NaN with 0
df_customers['Active'] = df_customers['Active'].fillna(0) #fill nulls/NaN with 0
df_customers['fashion_news_frequency'] = df_customers['fashion_news_frequency'].replace('None', np.NaN)
df_customers['fashion_news_frequency'] = df_customers['fashion_news_frequency'].replace('NONE', np.NaN)
df_customers['age'] = df_customers['age'].fillna(df_customers['age'].median())   #fill nulls/NaN with median age
df_customers.drop("postal_code", axis=1, inplace=True) #drop the postal code field

# one hot encoding
obj_cols = df_customers.select_dtypes("object").columns.to_list()
obj_cols.remove('customer_id')
df_cust_new = pd.get_dummies(df_customers, columns=obj_cols, drop_first=False) 

# view df
# df_cust_new.head()
# df_cust_new.info()
# df_cust_new.describe()

## Cleaning articles.csv
This dataset contains 25 features and 105,542 records.

In [48]:
# Load the data
df_articles = pd.read_csv('/Users/mythulam/Desktop/Masters/03_Fall_2022/02_Strategy_and_Performance_Management/Session_10_Group_Project/Data/articles.csv')

# extract columns to make new df
df_arti_new = df_articles[['article_id', 'index_name']]

# view new df
# df_arti_new.info()
# df_arti_new.head()

# merge new articles.csv with transactions.csv

In [49]:
# Load the data
df_transaction = pd.read_csv('/Users/mythulam/Desktop/Masters/03_Fall_2022/02_Strategy_and_Performance_Management/Session_10_Group_Project/Data/transactions_train.csv')

# copy df and merge to articles to get index name
df_trans_new = df_transaction.copy()
df_trans_new = df_trans_new.merge(df_arti_new, on='article_id', how='left', indicator=False)

# change date type
df_trans_new['t_dat'] = df_trans_new['t_dat'].astype("datetime64")

# view df
# df_trans_new.head()

In [50]:
df_trans_new.groupby(['index_name']).size()

index_name
Baby Sizes 50-98                    254137
Children Accessories, Swimwear      109191
Children Sizes 134-170              503507
Children Sizes 92-140               350514
Divided                            7138254
Ladies Accessories                 1774731
Ladieswear                        13058283
Lingeries/Tights                   5582246
Menswear                           1771053
Sport                              1246408
dtype: int64

## Aggregating Transaction Data by Customer ID

new columns:


*   Number of in-store transactions
*   Number of online transactions
*   Percent in-store purchases
*   Sum of total transaction prices

For all 10 index names:
*   Number of index name category transactions
*   Sum of index name category transactions prices (MAYBE)

For all 12 months:
*   Number of transactions per month
*   Sum of transaction prices per month (MAYBE)



In [51]:
# create first customer df based on transactions
df_cust_trans = df_trans_new.groupby(['customer_id','sales_channel_id']).size()
df_cust_trans = df_cust_trans.unstack()
df_cust_trans.rename(columns = {1:'num_instore', 2:'num_online'}, inplace = True)
df_cust_trans = df_cust_trans.fillna(0)

# add more columns
df_cust_trans['total_trans'] = df_cust_trans['num_instore'] + df_cust_trans['num_online']
df_cust_trans['perc_instore'] = df_cust_trans['num_instore'] / df_cust_trans['total_trans']
df_cust_trans['total_spent'] = df_trans_new.groupby('customer_id').sum()['price'].tolist()

#view df
df_cust_trans.head()

sales_channel_id,num_instore,num_online,total_trans,perc_instore,total_spent
customer_id,,,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,9.0,12.0,21.0,0.428571,0.648983
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,5.0,81.0,86.0,0.058140,2.601932
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,0.0,18.0,18.0,0.000000,0.704780
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,0.0,2.0,2.0,0.000000,0.060983
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,2.0,11.0,13.0,0.153846,0.469695


In [52]:
# dataframe of customer by index (number of transactions)
df_ct_index = df_trans_new.groupby(['customer_id','index_name']).size()
df_ct_index = df_ct_index.unstack()
df_ct_index.rename(columns = {'Ladieswear':'num_ladies', 'Lingeries/Tights':'num_lingeries','Baby Sizes 50-98':'num_bb50_98', 'Menswear': 'num_mens',
       'Ladies Accessories':'num_ladies_acc', 'Sport':'num_sport', 'Children Sizes 92-140':'num_child92_140', 'Divided':'num_divided',
       'Children Sizes 134-170':'num_child134_170', 'Children Accessories, Swimwear':'num_child_acc' }, inplace = True)
df_ct_index = df_ct_index.fillna(0)

# view df
df_ct_index.head()

index_name,num_bb50_98,num_child_acc,num_child134_170,num_child92_140,num_divided,num_ladies_acc,num_ladies,num_lingeries,num_mens,num_sport
customer_id,,,,,,,,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,0.0,1.0,2.0,2.0,2.0,0.0,13.0,0.0,1.0,0.0
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,4.0,0.0,0.0,0.0,19.0,6.0,20.0,36.0,0.0,1.0
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,0.0,0.0,0.0,0.0,1.0,0.0,6.0,6.0,4.0,1.0
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,0.0,2.0


In [53]:
# dataframe of customer by index (sum of price of transactions)
df_ct_indexp = df_trans_new.groupby(['customer_id','index_name']).sum()['price']
df_ct_indexp = df_ct_indexp.unstack()
df_ct_indexp.rename(columns = {'Ladieswear':'price_ladies', 'Lingeries/Tights':'price_lingeries','Baby Sizes 50-98':'price_bb50_98', 'Menswear': 'price_mens',
       'Ladies Accessories':'price_ladies_acc', 'Sport':'price_sport', 'Children Sizes 92-140':'price_child92_140', 'Divided':'price_divided',
       'Children Sizes 134-170':'price_child134_170', 'Children Accessories, Swimwear':'price_child_acc' }, inplace = True)
df_ct_indexp = df_ct_indexp.fillna(0)

# view df
df_ct_indexp.head()

index_name,price_bb50_98,price_child_acc,price_child134_170,price_child92_140,price_divided,price_ladies_acc,price_ladies,price_lingeries,price_mens,price_sport
customer_id,,,,,,,,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,0.000000,0.018288,0.060983,0.040644,0.052932,0.000000,0.440559,0.000000,0.035576,0.000000
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,0.089763,0.000000,0.000000,0.000000,0.826797,0.230407,0.603051,0.829898,0.000000,0.022017
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,0.000000,0.000000,0.000000,0.000000,0.030492,0.000000,0.225322,0.201593,0.186373,0.061000
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060983
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.246085,0.169407,0.000000,0.054203


In [35]:
# create new month column in transaction df
# df_trans_new['month'] = pd.DatetimeIndex(df_trans_new['t_dat']).month

# view df
# df_trans_new.head()

KeyboardInterrupt: 

In [ ]:
# dataframe of customer by month (total transactions)
# df_ct_month = df_trans_new.groupby(['customer_id','month']).size()
# df_ct_month = df_ct_month.unstack()
# df_ct_month.rename(columns = {1:'num_jan', 2:'num_feb', 3:'num_mar', 4:'num_apr', 5:'num_may', 6:'num_jun', 7:'num_jul', 8:'num_aug', 9:'num_sep', 10:'num_oct', 11:'num_nov', 12:'num_dec'}, inplace = True)
# df_ct_month = df_ct_month.fillna(0)

# view df
# df_ct_month.head()

In [ ]:
# dataframe of customer by month (total transactions)
# df_ct_monthp = df_trans_new.groupby(['customer_id','month']).sum()['price']
# df_ct_monthp = df_ct_monthp.unstack()
# df_ct_monthp.rename(columns = {1:'price_jan', 2:'price_feb', 3:'price_mar', 4:'price_apr', 5:'price_may', 6:'price_jun', 7:'price_jul', 8:'price_aug', 9:'price_sep', 10:'price_oct', 11:'price_nov', 12:'price_dec'}, inplace = True)
# df_ct_monthp = df_ct_monthp.fillna(0)

# view df
# df_ct_monthp.head()

In [54]:
# merge all transaction dataframes together
df_cust_trans = df_cust_trans.merge(df_ct_index, on='customer_id', how='left', indicator=False)
df_cust_trans = df_cust_trans.merge(df_ct_indexp, on='customer_id', how='left', indicator=False)
# df_cust_trans = df_cust_trans.merge(df_ct_month, on='customer_id', how='left', indicator=False)
# df_cust_trans = df_cust_trans.merge(df_ct_monthp, on='customer_id', how='left', indicator=False)

# view df
# df_cust_trans.info()
# df_cust_trans.head()

# merge customer and transaction together
df_cust_new = df_cust_new.merge(df_cust_trans, on='customer_id', how='left', indicator=False)
# fill na
df_cust_new = df_cust_new.fillna(0)

# view df
df_cust_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1371980 entries, 0 to 1371979
Data columns (total 34 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   customer_id                       1371980 non-null  object 
 1   FN                                1371980 non-null  float64
 2   Active                            1371980 non-null  float64
 3   age                               1371980 non-null  float64
 4   club_member_status_ACTIVE         1371980 non-null  uint8  
 5   club_member_status_LEFT CLUB      1371980 non-null  uint8  
 6   club_member_status_PRE-CREATE     1371980 non-null  uint8  
 7   fashion_news_frequency_Monthly    1371980 non-null  uint8  
 8   fashion_news_frequency_Regularly  1371980 non-null  uint8  
 9   num_instore                       1371980 non-null  float64
 10  num_online                        1371980 non-null  float64
 11  total_trans                       137

In [55]:
# export df to csv
df_cust_new.to_csv('/Users/mythulam/Desktop/Masters/03_Fall_2022/02_Strategy_and_Performance_Management/Session_10_Group_Project/Data/df_cust_new3.csv', encoding = 'utf-8-sig', index=False) 

We will use this csv with the churn prediction analysis results to inform our customer segmentation.